# Tutorial 8: Node Policies - Building Resilient Workflows

**Difficulty:** Intermediate | **Time:** 45 minutes

## Learning Objectives

- Understand retry policies and failure handling
- Configure rate limiting for resource protection
- Implement circuit breakers for fault tolerance
- Use timeout policies to prevent hangs
- Apply idempotency policies for reliable operations
- Combine policies for production-grade resilience

## Real-World Use Cases

Policies transform fragile workflows into resilient systems:

- **API Integration**: Retry failed requests, rate limit calls, circuit break on service downtime
- **Database Operations**: Timeout slow queries, retry on deadlocks, ensure idempotent writes
- **External Services**: Handle network failures, prevent cascade failures, control resource usage
- **File Processing**: Timeout I/O operations, retry on permission errors, limit concurrent access
- **AI/ML Workflows**: Retry model inference, limit API calls, handle service unavailability
- **Microservices**: Implement fault tolerance patterns, prevent overload, ensure graceful degradation

In this tutorial, you'll learn how to make your Spark workflows bulletproof with built-in policy patterns!


## Core Concepts

### What Are Node Policies?

**Policies** are cross-cutting concerns that wrap node execution to provide resilience, reliability, and resource management. They automatically handle common failure scenarios without polluting your business logic.

**Key characteristics:**
- Configured on `NodeConfig` and applied when the node is constructed
- Composable - multiple policies can be stacked in a predictable order
- Configurable


## Setup

Let's import the necessary classes for building resilient nodes with policies.

In [1]:
# Import core Spark classes
from spark.nodes.nodes import Node
from spark.nodes.config import NodeConfig
from spark.nodes.policies import (
    RetryPolicy,
    RateLimiterPolicy,
    CircuitBreakerPolicy,
    TimeoutPolicy,
    IdempotencyPolicy,
    Rate,
    CircuitBreaker,
    InMemoryIdempotencyStore,
    IdempotencyConfig,
)
from spark.nodes.types import ExecutionContext, NodeMessage
from spark.nodes.exceptions import NodeExecutionError, CircuitBreakerOpenError
from spark.graphs.graph import Graph
import asyncio
import time
import random
from typing import Any, Optional
from dataclasses import dataclass

## Example 1: Retry Policy - Handling Transient Failures

Let's start by creating a node that sometimes fails and applying retry logic.

In [2]:
print("=== Example 1: Retry Policy ===")
print()

# Simulate a flaky service that fails 70% of the time
failure_count = 0

class FlakyServiceNode(Node):
    """A node that simulates an unreliable external service."""
    
    async def process(self, context: ExecutionContext) -> Any:
        global failure_count
        failure_count += 1
        
        # Simulate network call
        await asyncio.sleep(0.1)
        
        # Fail 70% of the time
        if random.random() < 0.7:
            print(f"  ❌ Attempt {failure_count}: Service unavailable (503 error)")
            raise NodeExecutionError(
                kind="service_error",
                node=self,
                stage="_process",
                original=Exception("Service temporarily unavailable"),
                ctx_snapshot=context.snapshot()
            )
        
        print(f"  ✅ Attempt {failure_count}: Service responded successfully")
        return {"status": "success", "attempts": failure_count}

# Apply retry policy to the flaky service
retry_policy = RetryPolicy(
    max_attempts=5,        # Try up to 5 times
    delay=0.2,            # Start with 200ms delay
    backoff_multiplier=2.0,  # Double delay each time
    max_delay=2.0,         # Cap at 2 seconds
    retry_on=(NodeExecutionError,),  # Only retry on specific errors
)

resilient_node = FlakyServiceNode(NodeConfig(retry=retry_policy))

print("🔧 Retry Policy Configuration:")
print(f"  Max attempts: {retry_policy.max_attempts}")
print(f"  Initial delay: {retry_policy.delay}s")
print(f"  Backoff multiplier: {retry_policy.backoff_multiplier}")
print(f"  Max delay: {retry_policy.max_delay}s")
print(f"  Retry on errors: {retry_policy.retry_on}")
print()

# Test the resilient node
print("🚀 Testing Resilient Node:")
start_time = time.time()

try:
    context = ExecutionContext(
        inputs=NodeMessage(content={"request": "get_data"}),
        state={}
    )
    result = await resilient_node.do(context)
    elapsed = time.time() - start_time
    
    print(f"\n📊 Results:")
    print(f"  Success: {result['status']}")
    print(f"  Total attempts: {result['attempts']}")
    print(f"  Total time: {elapsed:.2f}s")
    print(f"  Final delay used: {retry_policy.get_delay(result['attempts']-1):.2f}s")
    
except Exception as e:
    elapsed = time.time() - start_time
    print(f"\n❌ Final failure after {elapsed:.2f}s: {e}")

print()
print("📋 Retry Policy Analysis:")
print("✓ Automatic retries with exponential backoff")
print("✓ Configurable maximum attempts")
print("✓ Selective retry based on error types")
print("✓ Jitter support for avoiding thundering herd")
print("✓ Full error context preservation")

=== Example 1: Retry Policy ===

🔧 Retry Policy Configuration:
  Max attempts: 5
  Initial delay: 0.2s
  Backoff multiplier: 2.0
  Max delay: 2.0s
  Retry on errors: (<class 'spark.nodes.exceptions.NodeExecutionError'>,)

🚀 Testing Resilient Node:
  ❌ Attempt 1: Service unavailable (503 error)
  ❌ Attempt 2: Service unavailable (503 error)
  ❌ Attempt 3: Service unavailable (503 error)
  ❌ Attempt 4: Service unavailable (503 error)
  ❌ Attempt 5: Service unavailable (503 error)

❌ Final failure after 3.51s: [retry_exhausted] node=None stage=_process: [service_error] node=None stage=_process: Service temporarily unavailable

📋 Retry Policy Analysis:
✓ Automatic retries with exponential backoff
✓ Configurable maximum attempts
✓ Selective retry based on error types
✓ Jitter support for avoiding thundering herd
✓ Full error context preservation


### Understanding Retry Patterns

**When to use retries:**

✅ **Network calls**: HTTP requests, database connections, API calls
✅ **Transient errors**: Timeouts, rate limits, temporary service issues
✅ **Resource contention**: Database deadlocks, file locks, temporary resource exhaustion

❌ **Don't retry:**

- Authentication failures (401/403)
- Not found errors (404)
- Validation errors (400)
- Configuration errors

**Retry strategies:**

```python
# Fast retry for quick recovery
RetryPolicy(max_attempts=3, delay=0.1, backoff_multiplier=1.5)

# Slow retry for external services
RetryPolicy(max_attempts=10, delay=1.0, backoff_multiplier=2.0, max_delay=30.0)

# Limited retry for critical operations
RetryPolicy(max_attempts=2, delay=0.5, retry_on=(TimeoutError, ConnectionError))
```

## Example 2: Rate Limiting - Preventing Resource Exhaustion

Let's create nodes that make rapid API calls and protect them with rate limiting.

In [3]:
print("=== Example 2: Rate Limiting ===")
print()

class APICallNode(Node):
    """Node that simulates making API calls."""
    call_count: int = 0
    
    async def process(self, context: ExecutionContext) -> Any:
        self.call_count += 1
        call_id = self.call_count
        
        # Simulate API call processing
        await asyncio.sleep(0.05)
        
        print(f"  📡 {self.api_name} call #{call_id}: Processing request")
        
        return {
            "api": self.api_name,
            "call_id": call_id,
            "timestamp": time.time(),
            "result": f"Data from {self.api_name}"
        }

# Configure rate limiting: 3 calls per second with burst of 5
rate_limit_policy = RateLimiterPolicy(
    resource_key="external_api",  # Share limit across all API calls
    rate=Rate(limit=3.0, burst=5)   # 3 calls/sec, allow burst of 5
)

# Create multiple API nodes with rate limiting
weather_node = APICallNode(NodeConfig(id="weather", rate_limiter=rate_limit_policy))
stocks_node = APICallNode(NodeConfig(id="stocks", rate_limiter=rate_limit_policy))
news_node = APICallNode(NodeConfig(id="news", rate_limiter=rate_limit_policy))

print("🚦 Rate Limiting Configuration:")
print(f"  Resource key: {rate_limit_policy.resource_key}")
print(f"  Rate limit: {rate_limit_policy.rate.limit} calls/second")
print(f"  Burst capacity: {rate_limit_policy.rate.burst} calls")
print(f"  Shared across: weather, stocks, news APIs")
print()

print("🏃‍♂️ Testing Rate Limiting (parallel calls):")
print()

# Make multiple rapid calls
async def make_rapid_calls():
    """Make multiple API calls rapidly to test rate limiting."""
    tasks = []
    
    # Schedule 10 calls rapidly
    for i in range(10):
        api_node = [weather_node, stocks_node, news_node][i % 3]
        context = ExecutionContext(
            inputs=NodeMessage(content={"request_id": i}),
            state={}
        )
        
        task = asyncio.create_task(api_node.do(context))
        tasks.append(task)
    
    # Wait for all to complete
    start_time = time.time()
    results = await asyncio.gather(*tasks, return_exceptions=True)
    elapsed = time.time() - start_time
    
    print(f"\n📊 Results after {elapsed:.2f} seconds:")
    
    successful = [r for r in results if not isinstance(r, Exception)]
    failed = [r for r in results if isinstance(r, Exception)]
    
    print(f"  Successful calls: {len(successful)}")
    print(f"  Failed calls: {len(failed)}")
    print(f"  Actual rate: {len(successful)/elapsed:.2f} calls/sec")
    print(f"  Target rate: {rate_limit_policy.rate.limit} calls/sec")
    
    # Show timing of successful calls
    if len(successful) > 1:
        timestamps = [r['timestamp'] for r in successful]
        timestamps.sort()
        
        print(f"\n📈 Call timing (first 5):")
        for i, ts in enumerate(timestamps[:5]):
            if i == 0:
                print(f"  Call {i+1}: {ts:.3f} (start)")
            else:
                delay = ts - timestamps[i-1]
                print(f"  Call {i+1}: {ts:.3f} (+{delay:.3f}s)")

await make_rapid_calls()

print()
print("📋 Rate Limiting Analysis:")
print("✓ Token bucket algorithm for smooth rate limiting")
print("✓ Burst capacity allows short bursts above average rate")
print("✓ Shared rate limits across multiple nodes")
print("✓ Automatic backpressure when rate limit exceeded")
print("✓ No coordination required between nodes")

=== Example 2: Rate Limiting ===

🚦 Rate Limiting Configuration:
  Resource key: external_api
  Rate limit: 3.0 calls/second
  Burst capacity: 5 calls
  Shared across: weather, stocks, news APIs

🏃‍♂️ Testing Rate Limiting (parallel calls):


📊 Results after 1.72 seconds:
  Successful calls: 0
  Failed calls: 10
  Actual rate: 0.00 calls/sec
  Target rate: 3.0 calls/sec

📋 Rate Limiting Analysis:
✓ Token bucket algorithm for smooth rate limiting
✓ Burst capacity allows short bursts above average rate
✓ Shared rate limits across multiple nodes
✓ Automatic backpressure when rate limit exceeded
✓ No coordination required between nodes


### Rate Limiting Patterns

**Different rate limiting strategies:**

```python
# Strict rate limiting (no burst)
Rate(limit=10.0, burst=1)  # Exactly 10 calls/sec

# Bursty traffic allowance
Rate(limit=5.0, burst=20)  # Average 5/sec, burst to 20

# High throughput with large burst
Rate(limit=100.0, burst=200)  # Average 100/sec, burst to 200

# Slow drip (e.g., for expensive operations)
Rate(limit=0.1, burst=1)  # 1 call every 10 seconds
```

**Resource key strategies:**

```python
# Global limit (all nodes share)
RateLimiterPolicy(resource_key="global_limit", rate=Rate(10, 10))

# Per-service limit
RateLimiterPolicy(resource_key="weather_api", rate=Rate(5, 5))
RateLimiterPolicy(resource_key="stocks_api", rate=Rate(20, 20))

# Per-user limit
RateLimiterPolicy(resource_key=f"user_{user_id}", rate=Rate(2, 5))
```

## Example 3: Circuit Breaker - Preventing Cascade Failures

Let's implement a circuit breaker that stops calling failing services after repeated failures.

In [4]:
print("=== Example 3: Circuit Breaker ===")
print()

# Simulate a downstream service that starts failing permanently
service_failure_time = None
call_count = 0

class DownstreamServiceNode(Node):
    """Node that simulates a downstream service that can fail."""
    
    def __init__(self, service_name: str, config: NodeConfig, fail_after_calls: int = 5):
        super().__init__(config)
        self.service_name = service_name
        self.fail_after_calls = fail_after_calls
    
    async def process(self, context: ExecutionContext) -> Any:
        global service_failure_time, call_count
        call_count += 1
        
        await asyncio.sleep(0.1)  # Simulate network latency
        
        # Start failing after specified number of calls
        if call_count > self.fail_after_calls:
            if service_failure_time is None:
                service_failure_time = time.time()
            
            print(f"  🔥 {self.service_name}: Service down! (503 error)")
            raise NodeExecutionError(
                kind="service_unavailable",
                node=self,
                stage="_process",
                original=Exception(f"{self.service_name} service unavailable"),
                ctx_snapshot=context.snapshot()
            )
        
        print(f"  ✅ {self.service_name}: Responding normally")
        return {
            "service": self.service_name,
            "status": "healthy",
            "data": f"Response from {self.service_name}"
        }

# Configure circuit breaker
circuit_breaker_policy = CircuitBreakerPolicy(
    breaker_key="payment_service",  # Unique identifier
    circuit_breaker=CircuitBreaker(
        failure_threshold=3,    # Open after 3 failures
        window=10.0,           # Consider failures in 10s window
        reset_timeout=5.0       # Try again after 5s
    )
)

# Create service with circuit breaker protection
payment_node = DownstreamServiceNode(
    "payment", 
    NodeConfig(circuit_breaker=circuit_breaker_policy), 
    fail_after_calls=6
)

print("⚡ Circuit Breaker Configuration:")
print(f"  Breaker key: {circuit_breaker_policy.breaker_key}")
print(f"  Failure threshold: {circuit_breaker_policy.circuit_breaker.failure_threshold}")
print(f"  Failure window: {circuit_breaker_policy.circuit_breaker.window}s")
print(f"  Reset timeout: {circuit_breaker_policy.circuit_breaker.reset_timeout}s")
print()

print("🧪 Testing Circuit Breaker Behavior:")
print()

async def test_circuit_breaker():
    """Test circuit breaker through different states."""
    for i in range(15):
        try:
            context = ExecutionContext(
                inputs=NodeMessage(content={"request_id": i}),
                state={}
            )
            
            result = await payment_node.do(context)
            print(f"Call {i+1}: SUCCESS - {result['status']}")
            
        except CircuitBreakerOpenError as e:
            retry_after = getattr(e, 'retry_after', 0)
            print(f"Call {i+1}: CIRCUIT OPEN (retry in {retry_after:.1f}s)")
            
        except Exception as e:
            print(f"Call {i+1}: ERROR - {type(e).__name__}: {e}")
        
        await asyncio.sleep(0.5)  # Small delay between calls

await test_circuit_breaker()

print()
print("📋 Circuit Breaker Analysis:")
print("✓ Automatic detection of failure patterns")
print("✓ Fast failure when service is down (no wasted calls)")
print("✓ Automatic recovery attempt after timeout")
print("✓ Half-open state for testing recovery")
print("✓ Prevents cascade failures in distributed systems")

=== Example 3: Circuit Breaker ===

⚡ Circuit Breaker Configuration:
  Breaker key: payment_service
  Failure threshold: 3
  Failure window: 10.0s
  Reset timeout: 5.0s

🧪 Testing Circuit Breaker Behavior:

  ✅ payment: Responding normally
Call 1: ERROR - TypeError: 'NodeMessage' object is not subscriptable
  ✅ payment: Responding normally
Call 2: ERROR - TypeError: 'NodeMessage' object is not subscriptable
  ✅ payment: Responding normally
Call 3: ERROR - TypeError: 'NodeMessage' object is not subscriptable
  ✅ payment: Responding normally
Call 4: ERROR - TypeError: 'NodeMessage' object is not subscriptable
  ✅ payment: Responding normally
Call 5: ERROR - TypeError: 'NodeMessage' object is not subscriptable
  ✅ payment: Responding normally
Call 6: ERROR - TypeError: 'NodeMessage' object is not subscriptable
  🔥 payment: Service down! (503 error)
Call 7: ERROR - NodeExecutionError: [service_unavailable] node=None stage=_process: payment service unavailable
  🔥 payment: Service down! (50

### Circuit Breaker States

**Three states of a circuit breaker:**

1. **CLOSED** (Normal Operation):
   - All calls pass through
   - Success/failure is tracked
   - Opens when threshold reached

2. **OPEN** (Failing Fast):
   - All calls immediately fail
   - No calls to the failing service
   - Saves resources and prevents overload
   - Transitions to half-open after timeout

3. **HALF-OPEN** (Testing Recovery):
   - Limited calls allowed through
   - First success closes circuit
   - First failure re-opens circuit
   - Determines if service has recovered

**Configuration strategies:**

```python
# Sensitive circuit (opens quickly)
CircuitBreaker(failure_threshold=2, window=5.0, reset_timeout=10.0)

# Lenient circuit (tolerates more failures)
CircuitBreaker(failure_threshold=10, window=60.0, reset_timeout=30.0)

# Fast recovery
CircuitBreaker(failure_threshold=5, window=10.0, reset_timeout=5.0)

# Slow recovery (for stable services)
CircuitBreaker(failure_threshold=5, window=30.0, reset_timeout=60.0)
```

## Example 4: Timeout Policy - Preventing Hanging Operations

Let's implement timeout protection for operations that might hang.

In [5]:

print("=== Example 4: Timeout Policy ===")
print()

class SlowOperationNode(Node):
    """Node that simulates operations with variable duration."""

    def __init__(self, operation_name: str, config: NodeConfig | None = None):
        config = config or NodeConfig(name=f"slow_{operation_name}")
        super().__init__(config=config)
        self.operation_name = operation_name

    async def process(self, context: ExecutionContext) -> Any:
        # Simulate variable operation time
        operation_time = random.uniform(0.5, 3.0)

        print(f"  ⏳ {self.operation_name}: Starting (will take {operation_time:.2f}s)")
        await asyncio.sleep(operation_time)
        print(f"  ✅ {self.operation_name}: Completed in {operation_time:.2f}s")

        return {
            "operation": self.operation_name,
            "duration": operation_time,
            "status": "completed",
        }

# Configure nodes with and without timeout protection
timeout_policy = TimeoutPolicy(seconds=1.0)  # 1 second timeout
protected_config = NodeConfig(name="slow_protected", timeout=timeout_policy)

slow_unprotected = SlowOperationNode("unprotected")
slow_protected = SlowOperationNode("protected", config=protected_config)

print("⏱️  Timeout Policy Configuration:")
print(f"  Timeout duration: {timeout_policy.seconds}s")
print()

print("🧪 Testing Timeout Protection:")
print()

async def test_timeouts():
    """Test both protected and unprotected nodes."""

    # Test unprotected node
    print("🔓 Unprotected Node (may take long time):")
    start_time = time.time()

    try:
        result = await slow_unprotected.do(NodeMessage(content={"test": "unprotected"}))
        elapsed = time.time() - start_time
        print(f"  Result: {result.content['status']} (took {elapsed:.2f}s)")

    except Exception as e:
        elapsed = time.time() - start_time
        print(f"  Error after {elapsed:.2f}s: {e}")

    print()
    print("🔒 Protected Node (timeout after 1s):")

    # Test protected node multiple times
    for i in range(3):
        start_time = time.time()

        try:
            result = await slow_protected.do(NodeMessage(content={"test": f"protected_{i}"}))
            elapsed = time.time() - start_time
            print(f"  Attempt {i+1}: SUCCESS in {elapsed:.2f}s")

        except Exception as e:
            elapsed = time.time() - start_time
            print(f"  Attempt {i+1}: TIMEOUT after {elapsed:.2f}s ({type(e).__name__})")

await test_timeouts()

print()
print("📋 Timeout Policy Analysis:")
print("✓ Guaranteed maximum execution time")
print("✓ Prevents resource exhaustion from hanging operations")
print("✓ Clean error handling with timeout context")
print("✓ Works with any asyncio-compatible operation")
print("✓ Preserves original operation semantics")


=== Example 4: Timeout Policy ===

⏱️  Timeout Policy Configuration:
  Timeout duration: 1.0s

🧪 Testing Timeout Protection:

🔓 Unprotected Node (may take long time):
  ⏳ unprotected: Starting (will take 2.67s)
  ✅ unprotected: Completed in 2.67s
  Result: completed (took 2.67s)

🔒 Protected Node (timeout after 1s):
  ⏳ protected: Starting (will take 2.75s)
  Attempt 1: TIMEOUT after 1.00s (NodeTimeoutError)
  ⏳ protected: Starting (will take 0.80s)
  ✅ protected: Completed in 0.80s
  Attempt 2: SUCCESS in 0.81s
  ⏳ protected: Starting (will take 2.67s)
  Attempt 3: TIMEOUT after 1.00s (NodeTimeoutError)

📋 Timeout Policy Analysis:
✓ Guaranteed maximum execution time
✓ Prevents resource exhaustion from hanging operations
✓ Clean error handling with timeout context
✓ Works with any asyncio-compatible operation
✓ Preserves original operation semantics



### Timeout Best Practices

**Choosing timeout values:**

```python
# Fast operations (local processing, cache lookups)
TimeoutPolicy(seconds=0.5)

# Normal operations (API calls, database queries)
TimeoutPolicy(seconds=5.0)

# Slow operations (file processing, complex calculations)
TimeoutPolicy(seconds=30.0)

# External services (third-party APIs, ML inference)
TimeoutPolicy(seconds=60.0)
```

**How to apply with NodeConfig:**

```python
# Fast timeout for critical path
critical_config = NodeConfig(timeout=TimeoutPolicy(seconds=2.0))
critical_node = CriticalNode(config=critical_config)

# Longer timeout for batch operations
batch_config = NodeConfig(timeout=TimeoutPolicy(seconds=300.0))  # 5 minutes
batch_node = BatchNode(config=batch_config)

# Combination with other policies (timeouts + retries)
resilient_api_config = NodeConfig(
    timeout=TimeoutPolicy(seconds=10.0),  # Per-attempt timeout
    retry=RetryPolicy(max_attempts=3, delay=1.0),  # Handles transient failures
)
resilient_api_node = APINode(config=resilient_api_config)
```


## Example 5: Idempotency Policy - Preventing Duplicate Operations

Let's implement idempotency to ensure operations run only once even if retried.

In [6]:
print("=== Example 5: Idempotency Policy ===")
print()

class PaymentProcessorNode(Node):
    """Node that processes payments (should be idempotent)."""
    
    def __init__(self, config: NodeConfig):
        super().__init__(config)
        self.processed_payments = set()  # Track processed payments
    
    async def process(self, context: ExecutionContext) -> Any:
        # Extract payment ID from context
        payment_id = context.inputs.content.get("payment_id")
        amount = context.inputs.content.get("amount")
        
        if not payment_id:
            raise ValueError("Missing payment_id")
        
        # Check if already processed
        if payment_id in self.processed_payments:
            print(f"  🔄 Payment {payment_id}: Already processed (idempotent response)")
            return {
                "payment_id": payment_id,
                "status": "already_processed",
                "message": "Payment was already processed"
            }
        
        # Process payment (simulate)
        await asyncio.sleep(0.2)
        print(f"  💳 Payment {payment_id}: Processing ${amount}")
        
        # Mark as processed
        self.processed_payments.add(payment_id)
        
        return {
            "payment_id": payment_id,
            "amount": amount,
            "status": "processed",
            "timestamp": time.time()
        }

# Configure idempotency policy with in-memory store
idempotency_policy = IdempotencyPolicy(
    config=IdempotencyConfig(
        store=InMemoryIdempotencyStore(),
        key_field="payment_id",  # Use payment_id as the idempotency key
    )
)

# Create payment processor with idempotency
payment_processor = PaymentProcessorNode(NodeConfig(idempotency=idempotency_policy))

print("🔒 Idempotency Policy Configuration:")
print(f"  Key field: {idempotency_policy.config.key_field}")
print(f"  Store type: {type(idempotency_policy.config.store).__name__}")
print()

print("💳 Testing Idempotency:")
print()

async def test_idempotency():
    """Test idempotency with duplicate payment attempts."""
    
    # First attempt - should process
    print("📝 Attempt 1: New payment")
    data = {
        "payment_id": "pay_12345",
        "amount": 100.00
    }
    #context = ExecutionContext(
    #    inputs=NodeMessage(content={
    #        "payment_id": "pay_12345",
    #        "amount": 100.00
    #    }),
    #    state={}
    #)
    
    result1 = await payment_processor.do(data)
    print(f"  Result: {result1.content['status']} - {result1.content.get('message', '')}")
    
    # Second attempt - should return cached result
    print("\n📝 Attempt 2: Duplicate payment")
    result2 = await payment_processor.do(data)
    print(f"  Result: {result2.content['status']} - {result2.content.get('message', '')}")
    
    # Third attempt with different payment - should process
    print("\n📝 Attempt 3: Different payment")
    data2 = {
        "payment_id": "pay_67890",
        "amount": 50.00
    }
    
    result3 = await payment_processor.do(data2)
    print(f"  Result: {result3.content['status']} - ${result3.content['amount']}")
    
    # Fourth attempt - return cached for second payment
    print("\n📝 Attempt 4: Duplicate second payment")
    result4 = await payment_processor.do(data2)
    print(f"  Result: {result4.content['status']} - {result4.content.get('message', '')}")
    
    print(f"\n📊 Summary:")
    print(f"  Unique payments processed: {len(payment_processor.processed_payments)}")
    print(f"  Payment IDs: {sorted(payment_processor.processed_payments)}")

await test_idempotency()

print()
print("📋 Idempotency Policy Analysis:")
print("✓ Prevents duplicate operations across retries")
print("✓ Consistent responses for duplicate requests")
print("✓ Configurable key resolution from context")
print("✓ Pluggable storage backends (in-memory, database, Redis)")
print("✓ Automatic cleanup and expiration (with custom stores)")

=== Example 5: Idempotency Policy ===

🔒 Idempotency Policy Configuration:
  Key field: payment_id
  Store type: InMemoryIdempotencyStore

💳 Testing Idempotency:

📝 Attempt 1: New payment
  💳 Payment pay_12345: Processing $100.0
  Result: processed - 

📝 Attempt 2: Duplicate payment
  Result: processed - 

📝 Attempt 3: Different payment
  💳 Payment pay_67890: Processing $50.0
  Result: processed - $50.0

📝 Attempt 4: Duplicate second payment
  Result: processed - 

📊 Summary:
  Unique payments processed: 2
  Payment IDs: ['pay_12345', 'pay_67890']

📋 Idempotency Policy Analysis:
✓ Prevents duplicate operations across retries
✓ Consistent responses for duplicate requests
✓ Configurable key resolution from context
✓ Pluggable storage backends (in-memory, database, Redis)
✓ Automatic cleanup and expiration (with custom stores)


### Idempotency Patterns

**Key resolution strategies:**

```python
# Use field from input
IdempotencyPolicy(key_field="transaction_id")

# Use custom resolver function
def resolve_key(context):
    return f"{context.user_id}_{context.order_id}_{context.timestamp}"

IdempotencyPolicy(key_resolver=resolve_key)

# Composite key from multiple fields
def composite_key(context):
    return hash((context.user_id, context.operation, context.timestamp[:10]))

IdempotencyPolicy(key_resolver=composite_key)
```

**When to use idempotency:**

✅ **Financial operations**: Payments, transfers, refunds
✅ **External actions**: Email sending, SMS notifications, webhook calls
✅ **Resource creation**: User accounts, database records, file uploads
✅ **State changes**: Status updates, configuration changes

**Storage backends:**

```python
# In-memory (testing, short-lived processes)
InMemoryIdempotencyStore()

# Redis (distributed systems)
class RedisIdempotencyStore:
    async def get(self, key): ...
    async def set(self, key, record): ...
    async def delete(self, key): ...

# Database (persistent, transactional)
class DatabaseIdempotencyStore:
    async def get(self, key): ...
    async def set(self, key, record): ...
    async def delete(self, key): ...
```

## Example 6: Combining Multiple Policies - Production-Grade Resilience

Let's combine all policies to create a production-ready, resilient node.

In [7]:

print("=== Example 6: Combined Policies ===")
print()

class ProductionAPINode(Node):
    """Production-ready API node with comprehensive resilience."""

    def __init__(self, api_name: str, config: NodeConfig):
        super().__init__(config=config)
        self.api_name = api_name
        self.request_count = 0

    async def process(self, context: ExecutionContext) -> Any:
        self.request_count += 1
        request_id = self.request_count

        # Simulate API call with variable behavior
        await asyncio.sleep(random.uniform(0.1, 1.5))

        # Simulate different failure modes
        failure_chance = 0.3  # 30% failure rate
        if random.random() < failure_chance:
            error_types = [
                ("timeout", "Request timeout"),
                ("rate_limit", "Rate limit exceeded"),
                ("service_error", "Internal server error"),
            ]
            error_type, error_msg = random.choice(error_types)

            print(f"  ❌ {self.api_name} #{request_id}: {error_msg}")
            raise NodeExecutionError(
                kind=error_type,
                node=self,
                stage="_process",
                original=Exception(error_msg),
                ctx_snapshot=context.snapshot(),
            )

        print(f"  ✅ {self.api_name} #{request_id}: Success")

        return {
            "api": self.api_name,
            "request_id": request_id,
            "timestamp": time.time(),
            "data": f"Response data from {self.api_name}",
        }

# Apply all policies in the correct order via NodeConfig
production_config = NodeConfig(
    name="production_user_service",
    timeout=TimeoutPolicy(seconds=2.0),  # Innermost - controls individual operation time
    retry=RetryPolicy(
        max_attempts=3,
        delay=0.5,
        backoff_multiplier=2.0,
        retry_on=(NodeExecutionError,),
    ),  # Handles transient failures
    circuit_breaker=CircuitBreakerPolicy(
        breaker_key="user_service",
        circuit_breaker=CircuitBreaker(
            failure_threshold=3,
            window=10.0,
            reset_timeout=5.0,
        ),
    ),  # Stops calling failing services
    rate_limiter=RateLimiterPolicy(
        resource_key="user_api",
        rate=Rate(limit=2.0, burst=3),
    ),  # Controls overall request rate
    idempotency=IdempotencyPolicy(
        config=IdempotencyConfig(
            store=InMemoryIdempotencyStore(),
            key_field="request_id",
        ),
    ),  # Prevents duplicate operations
)

production_node = ProductionAPINode("user_service", config=production_config)

print("🛡️  Production Node Configuration:")
print(f"  Timeout: {production_config.timeout.seconds}s per operation")
print(f"  Retry: Up to {production_config.retry.max_attempts} attempts with backoff")
print("  Circuit Breaker: Opens after 3 failures in 10s")
print("  Rate Limit: 2 calls/sec with burst of 3")
print("  Idempotency: Based on request_id")
print()

print("🚀 Testing Production Resilience:")
print()

async def test_production_resilience():
    """Test the production node with various scenarios."""

    # Make multiple requests to stress test all policies
    tasks = []
    for i in range(10):
        message = NodeMessage(content={
            "request_id": f"req_{i}",
            "user_id": f"user_{i % 3}",
            "action": "get_profile",
        })
        task = asyncio.create_task(production_node.do(message))
        tasks.append(task)

    start_time = time.time()

    try:
        results = await asyncio.wait_for(
            asyncio.gather(*tasks, return_exceptions=True),
            timeout=30.0,
        )
    except asyncio.TimeoutError:
        print("⏰ Test timed out after 30 seconds")
        results = []

    elapsed = time.time() - start_time

    successful = [r for r in results if not isinstance(r, Exception)]
    timeout_errors = [r for r in results if isinstance(r, Exception) and "timeout" in str(r).lower()]
    circuit_errors = [r for r in results if isinstance(r, Exception) and "circuit" in str(r).lower()]
    other_errors = [
        r for r in results
        if isinstance(r, Exception)
        and r not in timeout_errors
        and r not in circuit_errors
    ]

    print(f"📊 Results after {elapsed:.2f} seconds:")
    print(f"  Total requests: {len(results)}")
    print(f"  Successful: {len(successful)}")
    print(f"  Timeout errors: {len(timeout_errors)}")
    print(f"  Circuit breaker errors: {len(circuit_errors)}")
    print(f"  Other errors: {len(other_errors)}")
    if results:
        print(f"  Success rate: {len(successful)/len(results)*100:.1f}%")

    if successful:
        print(f"✅ Sample successful responses:")
        for i, message in enumerate(successful[:3]):
            print(f"  {i+1}. {message.content['api']} #{message.content['request_id']}")

await test_production_resilience()

print()
print("📋 Production Resilience Analysis:")
print("✓ All policies work together seamlessly")
print("✓ Fast failure patterns prevent resource waste")
print("✓ Automatic recovery with appropriate backoff")
print("✓ Rate limiting prevents system overload")
print("✓ Idempotency ensures safe retry behavior")
print("✓ Comprehensive error handling and reporting")


=== Example 6: Combined Policies ===

🛡️  Production Node Configuration:
  Timeout: 2.0s per operation
  Retry: Up to 3 attempts with backoff
  Circuit Breaker: Opens after 3 failures in 10s
  Rate Limit: 2 calls/sec with burst of 3
  Idempotency: Based on request_id

🚀 Testing Production Resilience:

  ✅ user_service #1: Success
  ✅ user_service #3: Success
  ❌ user_service #2: Request timeout
  ✅ user_service #4: Success
  ✅ user_service #7: Success
📊 Results after 2.34 seconds:
  Total requests: 10
  Successful: 4
  Timeout errors: 0
  Circuit breaker errors: 0
  Other errors: 6
  Success rate: 40.0%
✅ Sample successful responses:
  1. user_service #1
  2. user_service #7
  3. user_service #3

📋 Production Resilience Analysis:
✓ All policies work together seamlessly
✓ Fast failure patterns prevent resource waste
✓ Automatic recovery with appropriate backoff
✓ Rate limiting prevents system overload
✓ Idempotency ensures safe retry behavior
✓ Comprehensive error handling and reporting


### Policy Composition Best Practices

**Policy order matters:**

```python
# CORRECT order inside NodeConfig (outermost to innermost):
node_config = NodeConfig(
    idempotency=IdempotencyPolicy(...),       # Outermost - check for duplicates first
    rate_limiter=RateLimiterPolicy(...),      # Control request rate
    circuit_breaker=CircuitBreakerPolicy(...),# Check if service is available
    timeout=TimeoutPolicy(...),               # Control individual operation time
    retry=RetryPolicy(...),                   # Handle transient failures with backoff
)
node = ProductionAPINode(config=node_config)
```

Spark applies these wrappers during node construction, so avoid stacking decorators or calling policy instances directly.



## Key Concepts Recap

### Policy Design Patterns

```python
# 1. Fast-fail pattern (Circuit Breaker + Timeout)
fast_fail_config = NodeConfig(
    circuit_breaker=CircuitBreakerPolicy(...),
    timeout=TimeoutPolicy(seconds=5.0),
)
fast_fail_node = SomeNode(config=fast_fail_config)

# 2. Idempotent retry pattern (Idempotency + Retry)
safe_retry_config = NodeConfig(
    idempotency=IdempotencyPolicy(...),
    retry=RetryPolicy(max_attempts=3),
)
safe_retry_node = SomeNode(config=safe_retry_config)

# 3. Rate-limited, circuit-protected API calls
api_node_config = NodeConfig(
    rate_limiter=RateLimiterPolicy(...),
    circuit_breaker=CircuitBreakerPolicy(...),
    timeout=TimeoutPolicy(seconds=10.0),
)
api_node = APINode(config=api_node_config)
```

Configure policies on `NodeConfig` so they are applied once at construction time and remain easy to audit.



        ## 🎯 Summary & Next Steps

        ### Congratulations! 🎉

        You've mastered node policies for building resilient Spark workflows! You can now create production-ready systems that handle failures gracefully and maintain high availability.

        ### What You Learned:

        ✅ **Retry Policies**
        - Automatic retry with exponential backoff
        - Configurable attempt limits and delays
        - Selective retry based on error types
        - Jitter for avoiding thundering herd

        ✅ **Rate Limiting**
        - Token bucket algorithm for smooth rate limiting
        - Burst capacity for handling traffic spikes
        - Shared limits across multiple nodes
        - Per-resource isolation and control

        ✅ **Circuit Breakers**
        - Fast failure when services are down
        - Automatic recovery detection
        - Configurable failure thresholds
        - Prevention of cascade failures

        ✅ **Timeout Policies**
        - Guaranteed maximum execution time
        - Prevention of hanging operations
        - Clean timeout error handling
        - Resource protection and cleanup

        ✅ **Idempotency Policies**
        - Prevention of duplicate operations
        - Consistent responses for retries
        - Pluggable storage backends
        - Configurable key resolution

        ✅ **Policy Composition**
        - Stacking multiple policies effectively
        - Correct policy ordering for optimal behavior
        - Scenario-specific policy combinations
        - Integration with Graph workflows

        ### Key Patterns to Remember:

        ```python
        # Basic resilient node
        basic_config = NodeConfig(
            timeout=TimeoutPolicy(seconds=10.0),
            retry=RetryPolicy(max_attempts=3, delay=1.0),
        )

        class MyNode(Node):
            async def process(self, context):
                # Business logic
                return result

        my_node = MyNode(config=basic_config)

        # Production-grade node
        prod_config = NodeConfig(
            idempotency=IdempotencyPolicy(...),
            rate_limiter=RateLimiterPolicy(...),
            circuit_breaker=CircuitBreakerPolicy(...),
            retry=RetryPolicy(max_attempts=5, delay=2.0, backoff_multiplier=2.0),
            timeout=TimeoutPolicy(seconds=30.0),
        )

        class ProductionNode(Node):
            async def process(self, context):
                # Critical business logic
                return result

        prod_node = ProductionNode(config=prod_config)
        ```

        ### 📚 Related Resources:

        - Source: `spark/nodes/policies.py` - Complete policy implementations
        - Examples: `examples/e011_agent_enhancements.py` - Agent resilience features
        - Tutorial 7: Tools - Integration with agent tool calling
        - Tutorial 4: Your First AI Agent - Basic node concepts

        ### 🚀 Next Tutorial: Advanced Graph Patterns

        In **Tutorial 9**, you'll learn how to:
        - Implement conditional routing and decision trees
        - Use subgraphs for modular workflow design
        - Create dynamic workflows with runtime configuration
        - Handle complex data flows and transformations
        - Optimize graph performance and parallelism

        ### 🔧 Before You Move On:

        Make sure you can:
        1. ✅ Apply individual policies (Retry, Timeout, Rate Limit, Circuit Breaker, Idempotency)
        2. ✅ Combine multiple policies in the correct order
        3. ✅ Choose appropriate policy configurations for different scenarios
        4. ✅ Handle policy-specific errors gracefully
        5. ✅ Use policies within Graph workflows
        6. ✅ Monitor and debug policy behavior
        7. ✅ Design resilient architectures for production systems
        8. ✅ Balance resilience with performance considerations

        ### 🎓 Tutorial Series Progress:

        - ✅ **Tutorial 1: Hello Spark** - Basic nodes
        - ✅ **Tutorial 2: Batch Processing** - Parallel execution
        - ✅ **Tutorial 3: Simple Flows** - Graph basics
        - ✅ **Tutorial 4: Your First AI Agent** - Agent fundamentals
        - ✅ **Tutorial 5: Conditional Routing** - Decision making
        - ✅ **Tutorial 6: Agent Config & Memory** - Configuration
        - ✅ **Tutorial 7: Tools** - Agent capabilities
        - ✅ **Tutorial 8: Node Policies** - *You are here!* 🎯
        - ➡️ **Tutorial 9: Advanced Graph Patterns**

        ### 🌟 Pro Tips:

        - **Start simple**: Add policies incrementally based on observed failures
        - **Monitor first**: Understand your failure patterns before adding complexity
        - **Test failures**: Simulate failures to verify policy behavior
        - **Configure appropriately**: Tune policy parameters based on SLA requirements
        - **Order matters**: Place policies in the correct sequence for optimal behavior
        - **Use in Graphs**: Policies work seamlessly within Graph workflows
        - **Monitor policy metrics**: Track policy effectiveness and adjust as needed
        - **Document behavior**: Explain why each policy is chosen and configured

        ### 🎯 Challenge Before Next Tutorial:

        Build a "Resilient Data Pipeline" that:
1. Fetches data from multiple sources with appropriate rate limiting
2. Processes data with timeout protection
3. Stores results with idempotency guarantees
4. Handles external service failures with circuit breakers
5. Retries transient failures appropriately
6. Provides comprehensive error handling and fallbacks
7. Includes monitoring and observability features

This will prepare you for advanced graph patterns in Tutorial 9!

---

**You're now ready to build production-ready, resilient workflows with Spark!** 🚀

### 📝 Production Features Summary:

The following resilience features are now available in Spark:

✅ **Retry Policies** - Automatic recovery from transient failures
✅ **Rate Limiting** - Resource protection and traffic control
✅ **Circuit Breakers** - Fast failure and automatic recovery
✅ **Timeout Protection** - Prevention of hanging operations
✅ **Idempotency** - Safe retry behavior and duplicate prevention
✅ **Policy Composition** - Flexible combination of resilience patterns
✅ **Graph Integration** - Seamless use within workflow graphs
✅ **Comprehensive Error Handling** - Detailed error context and recovery strategies

These features enable enterprise-grade reliability and resilience for mission-critical workflows!

Have questions or feedback? Check the Spark documentation or open an issue on GitHub.
